In [1]:
import requests 
import pandas as pd 
import numpy as np
import random 

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cryptography-2.4.2         |   py36h1ba5d50_0         618 KB
    openssl-1.1.1a             |    h14c3975_1000         4.0 MB  conda-forge
    libarchive-3.3.3           |       h5d8350f_5         1.5 MB
    grpcio-1.16.1              |   py36hf8bcb03_1         1.1 MB
    conda-4.6.2                |           py36_0         869 KB  conda-forge
    libssh2-1.8.0              |                1         239 KB  conda-forge
    python-3.6.8               |       h0371630_0        34.4 MB
    ------------------------------------------------------------
                                           Total:        42.6 MB

The following packages will be UPDATED:

    conda:        4.5.12-py36_1000         conda

In [2]:
toronto=pd.read_csv('table.csv')
toronto.head()

,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
CLIENT_ID = 'BAO51TXGNACUNPQAEIWIIRNUMU0F50FWYO1FNZBA2MTVRDHO' 
CLIENT_SECRET = '4VYVGIINUK4N2HHD2JHZUXGODKH345ONWLVYUD3TDO01VG4M' 
VERSION = '20180604'
LIMIT = 1
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BAO51TXGNACUNPQAEIWIIRNUMU0F50FWYO1FNZBA2MTVRDHO
CLIENT_SECRET:4VYVGIINUK4N2HHD2JHZUXGODKH345ONWLVYUD3TDO01VG4M


In [4]:
search_query = 'Restaurant'
radius = 500
LIMIT=1

prova=pd.DataFrame([])
for i in range(0,103):
    url= 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, toronto['Latitude'][i], toronto['Longitude'][i], VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    prova=prova.append(dataframe, sort=False)
prova.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.state,name,referralId,location.crossStreet,location.postalCode,venuePage.id
0,[],False,4b7054e3f964a5204d132de3,4379 Kingston Road,CA,Toronto,Canada,246,"[4379 Kingston Road, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.76535407, 'lng...",43.765354,-79.190536,ON,Wonder Season Chinese Restaurant,v-1549114597,NaN,NaN,NaN
0,"[{'id': '52af3ac83cf9994f4e043bf3', 'name': 'H...",False,4b1711a6f964a520cbc123e3,1920 Ellesmere Rd,CA,Scarborough,Canada,219,"[1920 Ellesmere Rd (at Bellamy Rd. N), Scarbor...","[{'label': 'display', 'lat': 43.77469659057996...",43.774697,-79.241142,ON,Federick Restaurant,v-1549114597,at Bellamy Rd. N,M1H 2V6,NaN
0,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4bf96c435317a593a23a017f,1225 Kennedy Rd,CA,Toronto,Canada,283,[1225 Kennedy Rd (at Forbes (Between Lawrence ...,"[{'label': 'display', 'lat': 43.75604153945313...",43.756042,-79.276276,ON,Karaikudi Chettinad South Indian Restaurant,v-1549114598,at Forbes (Between Lawrence and Ellesmere),M1P 4Y1,33638214
0,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",False,4b8e9db3f964a5201b2c33e3,2072 Lawrence Ave. E,CA,Toronto,Canada,477,"[2072 Lawrence Ave. E (at Warden Ave.), Toront...","[{'label': 'display', 'lat': 43.7460300445972,...",43.746030,-79.293843,ON,Wexford Restaurant,v-1549114599,at Warden Ave.,M1R 2Z5,NaN
0,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,565cf4eb498ee7e80e7bb715,55 Nugget Ave,CA,Scarborough,Canada,309,"[55 Nugget Ave, Scarborough ON M1S 3L1, Canada]","[{'label': 'display', 'lat': 43.792085, 'lng':...",43.792085,-79.259535,ON,Twilight Family Restaurant & Bar,v-1549114599,NaN,M1S 3L1,NaN


In [5]:
filtered_columns = ['name', 'categories'] + [col for col in prova.columns if col.startswith('location.')] + ['id']
dataframe_filtered = prova.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()


,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,state,crossStreet,postalCode,id
0,Wonder Season Chinese Restaurant,None,4379 Kingston Road,CA,Toronto,Canada,246,"[4379 Kingston Road, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.76535407, 'lng...",43.765354,-79.190536,ON,NaN,NaN,4b7054e3f964a5204d132de3
0,Federick Restaurant,Hakka Restaurant,1920 Ellesmere Rd,CA,Scarborough,Canada,219,"[1920 Ellesmere Rd (at Bellamy Rd. N), Scarbor...","[{'label': 'display', 'lat': 43.77469659057996...",43.774697,-79.241142,ON,at Bellamy Rd. N,M1H 2V6,4b1711a6f964a520cbc123e3
0,Karaikudi Chettinad South Indian Restaurant,Indian Restaurant,1225 Kennedy Rd,CA,Toronto,Canada,283,[1225 Kennedy Rd (at Forbes (Between Lawrence ...,"[{'label': 'display', 'lat': 43.75604153945313...",43.756042,-79.276276,ON,at Forbes (Between Lawrence and Ellesmere),M1P 4Y1,4bf96c435317a593a23a017f
0,Wexford Restaurant,Breakfast Spot,2072 Lawrence Ave. E,CA,Toronto,Canada,477,"[2072 Lawrence Ave. E (at Warden Ave.), Toront...","[{'label': 'display', 'lat': 43.7460300445972,...",43.746030,-79.293843,ON,at Warden Ave.,M1R 2Z5,4b8e9db3f964a5201b2c33e3
0,Twilight Family Restaurant & Bar,Restaurant,55 Nugget Ave,CA,Scarborough,Canada,309,"[55 Nugget Ave, Scarborough ON M1S 3L1, Canada]","[{'label': 'display', 'lat': 43.792085, 'lng':...",43.792085,-79.259535,ON,NaN,M1S 3L1,565cf4eb498ee7e80e7bb715


In [6]:
latitude=toronto['Latitude'][0]
longitude=toronto['Longitude'][0]
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
  
for i in range(0,103):
    latitude=toronto['Latitude'][i]
    longitude=toronto['Longitude'][i]
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=6,
        color='red',
        popup='Neighborhood',
        fill = True,
        fill_color = 'red',
        fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [7]:
finale=dataframe_filtered[['lat', 'lng']]

from sklearn.cluster import KMeans

kclusters = 5

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(finale)

kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 4, 1], dtype=int32)

In [8]:
finale.insert(0, 'Cluster Labels', kmeans.labels_)
finale.head()

,Cluster Labels,lat,lng
0,3,43.765354,-79.190536
0,3,43.774697,-79.241142
0,3,43.756042,-79.276276
0,3,43.746030,-79.293843
0,3,43.792085,-79.259535


In [9]:
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude=toronto['Latitude'][0]
longitude=toronto['Longitude'][0]
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lng, cluster in zip(finale['lat'], finale['lng'], finale['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters